# Hit identification - genome-wide CRIPSRa IFNG screen of T cells

In [ ]:
import lamindb as ln
import bionty as bt

ln.settings.transform.stem_uid = "PtTXoc0RbOIq"
ln.settings.transform.version = "2"

ln.track()

## Download assay readout from wetlab

In [ ]:
artifact = ln.Artifact.filter(
    description="CRIPSRa GWS IFNG readout 2023-08-25.csv"
).one()

In [ ]:
artifact.describe()

## Load in the assay data and perform analysis

In [ ]:
df_raw = artifact.load()

## Perform analysis and ingest results into LaminDB

Let's assume we performed analysis from raw data `df_raw`, now we generated two new data objects:
- df: a new df containing lots of columns of statistics.
- hits: a subsetted dataframe containing hits of the screen.

In [ ]:
df = df_raw.set_index("id")
df.head()

In [ ]:
hits_df = df[df["pos|fdr"] < 0.01].copy()

## Register the file of screen hits

In [ ]:
artifact_hits = ln.Artifact.from_df(
    hits_df, description="hits from schmidt22 crispra GWS"
)

In [ ]:
artifact_hits.save()

In [ ]:
efo = bt.ExperimentalFactor.lookup()
features = ln.Feature.lookup()

artifact_hits.labels.add(efo.grna_seq, features.assay)
artifact_hits.labels.add(efo.interferon_gamma, features.readout)

In [ ]:
is_experiment = ln.ULabel.filter(name="is_experiment").one()
is_biosample = ln.ULabel.filter(name="is_biosample").one()

exp1 = is_experiment.children.filter(description__contains="Genome-wide").one()
biosample = is_biosample.children.get(name="S001")

artifact_hits.labels.add(exp1, features.experiment)
artifact_hits.labels.add(biosample, features.biosample)

In [ ]:
artifact_hits.describe()

In [ ]:
artifact_hits.view_lineage()